In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc))
# print(inspect.getsource(apps.backward_geocode_slc))

# scene

In [3]:
dem_urlpath = "data/Rome-10m-DEM.tif"

dem_raster = scene.open_dem_raster(dem_urlpath, chunks=2500)
dem_raster

<xarray.DataArray 'band_data' (y: 5000, x: 5000)>
dask.array<getitem, shape=(5000, 5000), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
    spatial_ref  int64 ...

In [4]:
# _ = dem_raster.plot()

In [5]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 12.6 s, sys: 854 ms, total: 13.5 s
Wall time: 13.2 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 5000, x: 5000)>
dask.array<setitem, shape=(3, 5000, 5000), dtype=float64, chunksize=(3, 2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis         (axis) int64 0 1 2

# image

In [6]:
product_urlpath = (
    "data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE"
)
measurement_group = "IW3/VV"
orbit_group = f"{measurement_group}/orbit"
calibration_group = f"{measurement_group}/calibration"
output_urlpath = "Rome-10m-GTC-SLC.tif"

!ls -d {product_urlpath}

data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE


In [7]:
measurement = xr.open_dataarray(
    product_urlpath,
    engine="sentinel-1",
    group=measurement_group,
    chunks={"pixel": 2048},
)
measurement

<xarray.DataArray 'measurement' (line: 13671, pixel: 25061)>
dask.array<open_dataset-bcdce3ef36167bdb8f32becf58f72e2cmeasurement, shape=(13671, 25061), dtype=complex64, chunksize=(1519, 2048), chunktype=numpy.ndarray>
Coordinates:
  * pixel             (pixel) int64 0 1 2 3 4 ... 25056 25057 25058 25059 25060
  * line              (line) int64 0 1 2 3 4 5 ... 13666 13667 13668 13669 13670
    azimuth_time      (line) datetime64[ns] dask.array<chunksize=(13671,), meta=np.ndarray>
    slant_range_time  (pixel) float64 dask.array<chunksize=(2048,), meta=np.ndarray>
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  13.88904
    sar:pixel_spacing_range:    2.329562
    azimuth_time_interval:      0.002055556299999998
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    azimuth_steering_rate:      1.397440818
    number_of_bursts:           9
    lines_per_burst:            1519
    units:                      1
    long_name:                  digital number

In [8]:
calibration = xr.open_dataset(
    product_urlpath, engine="sentinel-1", group=calibration_group
)
beta_nought_lut = calibration.betaNought
beta_nought_lut

<xarray.DataArray 'betaNought' (line: 29, pixel: 628)>
array([[236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867],
       [236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867],
       [236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867],
       ...,
       [236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867],
       [236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867],
       [236.9867, 236.9867, 236.9867, ..., 236.9867, 236.9867, 236.9867]],
      dtype=float32)
Coordinates:
  * line     (line) int64 0 486 973 1636 2123 ... 13100 13765 14252 14738 15402
  * pixel    (pixel) int64 0 40 80 120 160 200 ... 24920 24960 25000 25040 25060
Attributes:
    units:      m m-1
    long_name:  beta nought calibration LUT

In [9]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 2.97 s, sys: 1.37 s, total: 4.35 s
Wall time: 4.35 s


<xarray.DataArray (line: 13671, pixel: 25061)>
dask.array<pow, shape=(13671, 25061), dtype=float32, chunksize=(1519, 2048), chunktype=numpy.ndarray>
Coordinates:
  * pixel             (pixel) int64 0 1 2 3 4 ... 25056 25057 25058 25059 25060
  * line              (line) int64 0 1 2 3 4 5 ... 13666 13667 13668 13669 13670
    azimuth_time      (line) datetime64[ns] dask.array<chunksize=(13671,), meta=np.ndarray>
    slant_range_time  (pixel) float64 dask.array<chunksize=(2048,), meta=np.ndarray>
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  13.88904
    sar:pixel_spacing_range:    2.329562
    azimuth_time_interval:      0.002055556299999998
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    azimuth_steering_rate:      1.397440818
    number_of_bursts:           9
    lines_per_burst:            1519
    units:                      m2 m-2
    long_name:                  beta nought

In [10]:
orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group)
position_ecef = orbit_ecef.position
position_ecef

<xarray.DataArray 'position' (axis: 3, azimuth_time: 16)>
array([[4657064.97853 , 4712297.916925, 4767003.903681, 4821176.38583 ,
        4874808.871124, 4927894.928873, 4980428.190747, 5032402.351598,
        5083811.170256, 5134648.470349, 5184908.141022, 5234584.137678,
        5283670.482715, 5332161.266259, 5380050.646874, 5427332.852286],
       [1776448.316703, 1777360.958901, 1777994.246465, 1778348.876228,
        1778425.58583 , 1778225.153543, 1777748.398105, 1776996.17854 ,
        1775969.393961, 1774668.983378, 1773095.92551 , 1771251.238588,
        1769135.980159, 1766751.246885, 1764098.174349, 1761177.936816],
       [5013314.106183, 4961243.291607, 4908612.381603, 4855427.302963,
        4801694.045627, 4747418.662004, 4692607.266299, 4637266.033804,
        4581401.200177, 4525019.060727, 4468125.969666, 4410728.339344,
        4352832.639528, 4294445.396682, 4235573.19325 , 4176222.66689 ]])
Coordinates:
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:10:21.029300 .....
  * axis          (axis) int64 0 1 2
Attributes:
    units:      m
    long_name:  ECEF position

In [11]:
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(position_ecef)
position_ecef = orbit_interpolator.position()
velocity_ecef = orbit_interpolator.velocity()
position_ecef

<xarray.DataArray 'position' (azimuth_time: 7501, axis: 3)>
array([[4657064.97861576, 1776448.31671965, 5013314.10621806],
       [4657175.96600368, 1776450.42124249, 5013210.5274712 ],
       [4657286.95131017, 1776452.52464528, 5013106.94646053],
       ...,
       [5427144.94061559, 1761190.14837911, 4176461.01298063],
       [5427238.89771613, 1761184.04313713, 4176341.84088746],
       [5427332.85236519, 1761177.93683131, 4176222.66690769]])
Coordinates:
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:10:21.029300 .....
  * axis          (axis) int64 0 1 2

In [12]:
%%time
dem_coords = geocoding.backward_geocode(dem_ecef, position_ecef, velocity_ecef)
dem_coords

CPU times: user 29.9 s, sys: 11.5 s, total: 41.4 s
Wall time: 28.3 s


<xarray.Dataset>
Dimensions:           (x: 5000, y: 5000, axis: 3)
Coordinates:
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 2500, 2500), meta=np.ndarray>

In [13]:
%%time
geocoded = xr.full_like(dem_raster, np.nan)

azimuth_time_min = dem_coords.azimuth_time.values.min()
azimuth_time_max = dem_coords.azimuth_time.values.max()
slant_range_time_min = dem_coords.slant_range_time.values.min()
slant_range_time_max = dem_coords.slant_range_time.values.max()

for burst_index in range(beta_nought.attrs["number_of_bursts"]):
    burst = xarray_sentinel.crop_burst_dataset(beta_nought, burst_index=burst_index)
    burst = burst.rolling(azimuth_time=2, slant_range_time=8).mean()
    if (
        burst.azimuth_time[-1] < azimuth_time_min
        or burst.azimuth_time[0] > azimuth_time_max
    ):
        continue
    if (
        burst.slant_range_time[-1] < slant_range_time_min
        or burst.slant_range_time[0] > slant_range_time_max
    ):
        continue
    # the `isel` is very crude way to remove the black bands in azimuth
    temp = burst.isel(azimuth_time=slice(30, -30)).interp(
        azimuth_time=dem_coords.azimuth_time,
        slant_range_time=dem_coords.slant_range_time,
        method="linear",
    )
    geocoded = xr.where(np.isfinite(temp), temp, geocoded)

geocoded

CPU times: user 15 s, sys: 8.39 s, total: 23.4 s
Wall time: 8.93 s


<xarray.DataArray (y: 5000, x: 5000)>
dask.array<where, shape=(5000, 5000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    pixel             (y, x) float64 1.78e+04 1.78e+04 ... 5.915e+03 5.912e+03
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06

In [14]:
# _ = geocoded.plot(vmax=1.0)

In [15]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)